# LangChainMemory
- 이전 대화 내용을 기억해서 문맥을 유지하는 역할 LangChain 0.3X 부터는 LCEL 기반으로 체인을 구성.
- RunnableWithMessageHistory, ChatMessageHistory 등의 컴포넌트를 활용하여 세션별 대화 기록을 관리
- 대화가 장기화될 경우 요약 메모리를 도입하여 과거 대화를 LLM으로 요약하고 축약된 형태로 저장해서 프롬프트의 길이 문제를 해결함.

In [23]:
%pip install --quiet langchain langchain-openai langchain-community python-dotenv langchain_redis

Note: you may need to restart the kernel to use updated packages.


In [24]:
from dotenv import load_dotenv
load_dotenv()

True

In [25]:
from langchain_core.chat_history import InMemoryChatMessageHistory
# 메모리 객체 생성
history = InMemoryChatMessageHistory()
history.add_user_message("안녕하세요. 제 이름은 홍길동입니다.")
history.add_ai_message("안녕하세요, 홍길동님! 만나서 반갑습니다.")

In [26]:
# 현재까지의 대화 내용 확인
for msg in history.messages:
    print(f"{msg.type}: {msg.content}")

human: 안녕하세요. 제 이름은 홍길동입니다.
ai: 안녕하세요, 홍길동님! 만나서 반갑습니다.


# Redis 기반 채팅 기록 저장소

In [27]:
from langchain_redis import RedisChatMessageHistory
import os

# Redis RUL 설정
# 포트 번호는 6379가 기본값입니다.
# 포트 번호가 다르다면 환경 변수 REDIS_URL에 redis://localhost:포트번호 형식으로 설정하세요.
# 환경 변수 REDIS_URL이 설정되어 있지 않으면 기본값으로 redis://localhost:6379 사용
REDIS_URL = os.getenv("REDIS_URL", "redis://localhost:6379")
session_id = "user_123"

# RedisChatMessageHistory 객체 생성
history = RedisChatMessageHistory(session_id=session_id, redis_url=REDIS_URL)
# Redis에 대화 내용 저장
history.add_user_message("안녕하세요. 제 이름은 홍길동입니다.")
history.add_ai_message("안녕하세요, 홍길동님! 만나서 반갑습니다.")

ConnectionError: Error 10061 connecting to localhost:6379. 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다.

# 세션 기반 다중 사용자 메모리 구조 구현 - 다중 사용자 챗봇
- 핵심: session_id를 키로 하는 메모리 저장소를 만들고, 사용자의 대화를 키 별로 저장한다.

In [39]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
# 프롬프트
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 뛰어난 한국어 상담 챗봇입니다. 질문에 친절하고 자세히 답변해주세요."),
    # history 키로 전달된 메시지 목록은 체인 실행 시 해당 위치에 넣겠다는 의미.
    MessagesPlaceholder(variable_name="history"), 
    ("human", "{input}")
])

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

In [40]:
# LCEL
from langchain_core.output_parsers import StrOutputParser
chain = prompt | llm | StrOutputParser()

In [41]:
# 세션별 메모리 저장소를 딕셔너리로 만들고, 존재하지 않는 새로운 세션 id가 들어오면 InMemoryChatMessageHistory를 생성
# get_session_history를 구현
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
# 세션 id -> 대화 기록 객체 매핑
store = {}
def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    """
    세션 ID에 해당하는 대화 기록 객체를 반환합니다. ( 없으면 새로 생성 )
    """    
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]
# 메모리를 통합한 체인 Wrapper 생성
chatbot = RunnableWithMessageHistory(
    runnable=chain,
    get_session_history=get_session_history,
    input_messages_key="input",
    history_messages_key="history"
)

In [42]:
# 두 개의 세션을 번갈아가면서 대화
# RunnableWithMessageHistory를 사용하여 세션별로 대화 기록을 관리합니다.
# 세션 ID를 사용하여 대화 기록을 구분하고, 각 세션에 대한 질문과 답변을 생성합니다.
sessions = ["user_a", "user_b"]
questions = [
    "안녕하세요, 저는 홍길동입니다. 당신은 누구신가요?",             # user_a의 첫 번째 질문
    "안녕하세요, 저는 이순신입니다. 당신은 어떤 일을 하시나요?",     # user_b의 첫 번째 질문
    "저는 프로그래밍을 배우고 있습니다. 당신은 어떤 일을 하시나요?",  # user_a의 두 번째 질문
    "저는 역사에 관심이 많습니다. 당신은 어떤 분야에 관심이 있나요?"  # user_b의 두 번째 질문
]
for idx, question in enumerate(questions):
    session_id = sessions[idx % len(sessions)]
    # 질문에 대한 답변 생성
    response = chatbot.invoke({"input": question}, config={"configurable": {"session_id": session_id}})
    print(f"Session {session_id} 질문: {question}")
    print(f"Session {session_id} 답변: {response}\n")

17:50:15 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Session user_a 질문: 안녕하세요, 저는 홍길동입니다. 당신은 누구신가요?
Session user_a 답변: 안녕하세요, 홍길동님! 저는 여러분의 질문에 답변하고 도움을 드리기 위해 만들어진 챗봇입니다. 어떤 궁금한 점이나 도움이 필요하신 부분이 있으신가요?

17:50:16 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Session user_b 질문: 안녕하세요, 저는 이순신입니다. 당신은 어떤 일을 하시나요?
Session user_b 답변: 안녕하세요, 이순신님! 저는 여러분의 질문에 답변하고, 다양한 정보와 도움을 제공하는 챗봇입니다. 궁금한 점이나 도움이 필요한 부분이 있다면 언제든지 말씀해 주세요. 역사, 과학, 문화, 일상생활 등 다양한 주제에 대해 이야기할 수 있습니다. 무엇을 도와드릴까요?

17:50:18 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Session user_a 질문: 저는 프로그래밍을 배우고 있습니다. 당신은 어떤 일을 하시나요?
Session user_a 답변: 프로그래밍을 배우고 계시다니 멋지네요! 저는 여러분의 질문에 답변하고, 정보 제공, 문제 해결, 그리고 다양한 주제에 대한 상담을 하는 역할을 하고 있습니다. 프로그래밍에 관련된 질문이나 도움이 필요하시면 언제든지 말씀해 주세요! 어떤 언어를 배우고 계신가요?

17:50:20 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK

In [43]:
result = chatbot.invoke({"input": "저는 철수에요. 반갑습니다."}, config={"configurable": {"session_id": "user_c"}})
print(f"Session user_c 질문: 저는 철수에요. 반갑습니다.")
print(f"Session user_c 답변: {result}\n")

17:50:21 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Session user_c 질문: 저는 철수에요. 반갑습니다.
Session user_c 답변: 안녕하세요, 철수님! 반갑습니다. 어떻게 도와드릴까요? 궁금한 점이나 이야기하고 싶은 것이 있다면 말씀해 주세요.



In [44]:
result = chatbot.invoke({"input": "저는 누구라고요?"}, config={"configurable": {"session_id": "user_a"}})
print(f"Session user_a 질문: 저는 누구라고요?")
print(f"Session user_a 답변: {result}\n")

17:50:22 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Session user_a 질문: 저는 누구라고요?
Session user_a 답변: 홍길동님이라고 말씀하셨습니다! 혹시 더 궁금한 점이나 다른 질문이 있으신가요? 도움이 필요하시면 언제든지 말씀해 주세요.



In [45]:
result = chatbot.invoke({"input": "저는 누구라고요?"}, config={"configurable": {"session_id": "user_b"}})
print(f"Session user_b 질문: 저는 누구라고요?")
print(f"Session user_b 답변: {result}\n")

17:50:24 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Session user_b 질문: 저는 누구라고요?
Session user_b 답변: 이순신님이라고 말씀하셨습니다! 이순신은 조선시대의 유명한 장군으로, 임진왜란 때의 전쟁 영웅으로 잘 알려져 있습니다. 그의 전략과 용기는 많은 사람들에게 영감을 주고 있습니다. 혹시 이순신 장군에 대해 더 이야기하고 싶으신가요, 아니면 다른 역사적 인물이나 사건에 대해 궁금한 점이 있으신가요?



In [46]:
result = chatbot.invoke({"input": "저는 누구라고요?"}, config={"configurable": {"session_id": "user_c"}})
print(f"Session user_c 질문: 저는 누구라고요?")
print(f"Session user_c 답변: {result}\n")

17:50:25 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Session user_c 질문: 저는 누구라고요?
Session user_c 답변: 철수님이라고 하셨습니다! 혹시 더 궁금한 점이나 다른 이야기를 나누고 싶으신가요? 언제든지 말씀해 주세요!



# 요약 메모리 구현( 대화 내용 자동 요약 )
```
긴 대화 내용을 모두 프롬프트에 기록하는 것은 비 효율적 -> 프롬프트의 길이 제한에 걸릴 가능성이 있음.

Conversation Summary Memory
0.3x 버전에서는 직접 요약용 체인을 만들어서 ChatMessageHistory에 적용
```

## 요약 방법
```
- 일정 길이 이상 대화가 누적되면, 과거 대화를 요약해서 핵심 내용만 남김
- 요약 결과를 메모리에 시스템 메시지 등으로 저장 -> 메모리 절약
- 새로운 사용자 입력 시 요약된 맥락 + 최근 몇 메시지만 참고하여 LLM에 전달
```

In [47]:
# 요약용 프롬프트 템플릿
summary_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 대화 요약 전문가입니다. 대화의 주요 내용을 간결하게 요약해주세요."),
    ("human", "{conversation}"), # 전체 대화 내용을 하나의 문자열로 전달
])
# LCEL
summary_chain = summary_prompt | llm | StrOutputParser()

In [48]:
# user_d 세션에 대화 내용을 기록.

# 긴 대화 내용을 생성
long_queries = [
    "안녕, 오늘 우리 뭐하려고 했지?",
    "아, 맞다. 내일 회의 자료를 준비해야지. 내일 회의는 몇 시지?",
    "그 회의에 누가 참석하는지 기억 나?",
    "단위 프로젝트 진행 상황도 공유해야 할까?",
    "최근에 이야기 했던 새로운 기능에 대한 업데이트는 있어?"
]

session_id = "user_d"
for q in long_queries:
    response = chatbot.invoke({"input": q}, config={"configurable": {"session_id": session_id}})
    print(f"Session {session_id} 질문: {q}")
    print(f"Session {session_id} 답변: {response}\n")

17:50:27 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Session user_d 질문: 안녕, 오늘 우리 뭐하려고 했지?
Session user_d 답변: 안녕하세요! 오늘 어떤 이야기를 나누고 싶으신가요? 궁금한 점이나 하고 싶은 주제가 있다면 말씀해 주세요. 함께 이야기해보아요!

17:50:28 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Session user_d 질문: 아, 맞다. 내일 회의 자료를 준비해야지. 내일 회의는 몇 시지?
Session user_d 답변: 회의 시간이 기억나지 않으신가요? 보통 회의 일정은 이메일이나 캘린더에 기록되어 있을 텐데요. 확인해보시면 좋을 것 같아요. 만약 회의 준비에 도움이 필요하시다면 어떤 자료를 준비해야 하는지 말씀해 주시면 도와드릴 수 있습니다!

17:50:31 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Session user_d 질문: 그 회의에 누가 참석하는지 기억 나?
Session user_d 답변: 회의 참석자에 대한 정보는 보통 회의 초대장이나 이메일에 포함되어 있을 텐데요. 그 내용을 확인해보시면 좋을 것 같습니다. 만약 참석자 목록을 정리해야 하거나, 특정 인물에 대해 궁금한 점이 있다면 말씀해 주세요. 도움이 될 수 있도록 최선을 다하겠습니다!

17:50:34 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Session user_d 질문: 단위 프로젝트 진행 상황도 공유해야 할까?
Session use

In [49]:
# 전체 대화 내용을 요약하고 마지막 사용자 질문-답변 쌍만 원본 유지
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
# 요약 대상 대화 내용 추출( 마지막 QA 쌍 제외한 이전 내용 )
message = store[session_id].messages

if len(message) > 2:
    original_dialogue = "\n".join([f"{msg.type.upper()}: {msg.content}" for msg in message[:-2]])
else:
    original_dialogue = "\n".join([f"{msg.type.upper()}: {msg.content}" for msg in message])

# llm으로 요약 생성
summary_text = summary_chain.invoke({"conversation": original_dialogue})
print("== 전체 대화 요약 ==")
print(f"Session {session_id} 전체 대화 요약: {summary_text}\n")

# 기존 메모리를 요약으로 교쳬: 이전 내용 요약본 + 최근 QA 유지
new_history = InMemoryChatMessageHistory()
new_history.messages.append(SystemMessage(content=f"요약: {summary_text}"))

# 최근 대화의 마지막 QA 쌍 유지
if len(message) > 2:
    last_user_message = message[-2]
    last_ai_message = message[-1]
    if isinstance(last_user_message, HumanMessage):
        new_history.messages.append(last_user_message.content)
    else:
        new_history.messages.append(last_user_message)
    if isinstance(last_ai_message, AIMessage):
        new_history.messages.append(last_ai_message.content)
    else:
        new_history.messages.append(last_ai_message)

# 새로운 메모리로 교체
store[session_id] = new_history

17:50:40 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
== 전체 대화 요약 ==
Session user_d 전체 대화 요약: 대화 요약: 
HUMAN은 내일 회의 자료를 준비해야 한다고 언급하며 회의 시간과 참석자를 기억하지 못한다고 말합니다. AI는 회의 일정과 참석자 정보를 이메일이나 캘린더에서 확인할 것을 제안하고, 단위 프로젝트 진행 상황을 공유하는 것이 좋다고 조언합니다. AI는 진행 상황을 시각적으로 정리하는 것도 효과적이라고 덧붙입니다.

